In [18]:
import XGBoostGRASP_HPO as hpo_grasp
import time
from phase2grasp import LocalSearch
from sklearn.datasets import load_breast_cancer, load_digits
import matplotlib.pyplot as plt

In [21]:
x_train, x_test, y_train, y_test = hpo_grasp.prepare_dataset(load_breast_cancer())

In [22]:
hyperparameter_ranges = {
    'n_estimators': (50, 500),
    'max_depth': (3, 10),
    'colsample_bytree': (0.5, 1),
    'reg_lambda': (0.01, 1.0),
    'subsample': (0.5, 1.0)
}

In [34]:
def run_it():
    start_build_time = time.time()
    best_intermediate_combinations = hpo_grasp.building_phase()
    end_build_time = time.time()
    
    first_score = hpo_grasp.evaluate_solution(best_intermediate_combinations.queue[0][2])
    
    ls = LocalSearch(hyperparameter_ranges)
    ls.set_fn(5)

    start_hill_time = time.time()
    outer_counter = 1
    #print(str(outer_counter) + "---------")
    #print()
    local_best_score, local_best_solution = hpo_grasp.hill_climb(best_intermediate_combinations.get()[2])
    while not best_intermediate_combinations.empty():
        outer_counter = outer_counter + 1
        print(str(outer_counter) + "---------")
        print()
        temporary_score, temporary_solution = hpo_grasp.hill_climb(best_intermediate_combinations.get()[2])
        if local_best_score < temporary_score:
            local_best_score = temporary_score
            local_best_solution = temporary_solution

    end_hill_time = time.time()
    #print("Hyperparameters: " + str(local_best_solution))
    #print("Achieved best score: " + str(local_best_score))
    xgboost_classifier = XGBClassifier(**local_best_solution)
    scores = cross_val_score(xgboost_classifier,x_train,y_train,cv=5,error_score='raise')
    #print(scores)
    #print(scores.mean(),scores.std())
    #print(str(end_build_time-start_build_time),str(end_hill_time-start_hill_time),str(end_hill_time-start_build_time))
    return first_score, local_best_score

In [ ]:
x=[]
y=[]
for i in range(20):
    x1, y1 = run_it()
    x.append(x1)
    y.append(y1)
plt.plot(x1,y1)

{'n_estimators': 500, 'max_depth': 3, 'colsample_bytree': 0.5, 'reg_lambda': 0.01, 'subsample': 0.5}
0.991207141671538
build finished
local search set to use fn 5
1---------

hill climb finished
Hyperparameters: {'n_estimators': 500, 'max_depth': 3, 'colsample_bytree': 0.5, 'reg_lambda': 0.01, 'subsample': 0.5}
Achieved best score: 0.991207141671538
[0.97802198 0.96703297 0.95604396 0.94505495 0.97802198]
0.9648351648351647 0.01281527888976988
11.102995872497559 4.996999740600586 16.166995763778687
{'n_estimators': 50, 'max_depth': 6, 'colsample_bytree': 1.0, 'reg_lambda': 0.505, 'subsample': 0.75}
0.9824561403508771
build finished
local search set to use fn 5
1---------

hill climb finished
Hyperparameters: {'n_estimators': 50, 'max_depth': 6, 'colsample_bytree': 1.0, 'reg_lambda': 0.505, 'subsample': 0.75}
Achieved best score: 0.9824561403508771
[0.97802198 0.96703297 0.95604396 0.93406593 0.97802198]
0.9626373626373625 0.01644684565614917
11.34000039100647 2.9609994888305664 14.3219